In [ ]:
import sys
sys.path.append('../../')

import estraces
import scared
import matplotlib.pyplot as plt
from kresca.plot import CurvePlot
from kresca.synchronization import PatternSynchronization

## Import Traces

In [ ]:
ths = estraces.read_ths_from_ets_file('../../Traces/ETS/Telkom35k.ets')

In [ ]:
print(ths)

## Curve Plot

In [ ]:
#put inline=True if you want to display on notebook
CurvePlot(ths.samples[:5, 357:], inline=True) 

## Make a Pattern

### Apply Filter to Traces

In [ ]:
#filter 
samples = ths.samples[2:4, 357:]
fs = 25e6
cutoff = 1e6
window_mean = 50
filtered_samples = scared.signal_processing.butterworth(samples, fs, cutoff)
prep_samples = scared.signal_processing.moving_mean(filtered_samples, window_size=window_mean)

In [ ]:
CurvePlot(prep_samples, inline=True) 

### Create Pattern from Samples

In [ ]:
start_pattern = 43000
end_pattern = 48000
reference_trace = 1
length = end_pattern - start_pattern

# pattern = ths[reference_trace].samples[start_pattern:end_pattern]

# use filtered traces
pattern = prep_samples[reference_trace, start_pattern:end_pattern]

In [ ]:
plt.figure(figsize=(16, 6))
# plt.plot(ths[reference_trace].samples, label='reference trace')
plt.plot(prep_samples[reference_trace], label='reference trace')
plt.plot(range(start_pattern, end_pattern), pattern, color='orange', label='pattern')
plt.xlim(start_pattern-length, end_pattern+length)
plt.legend(loc=2)
plt.show()

## Pattern Synchronization

In [ ]:
def low_pass_filter(data):
    return scared.signal_processing.butterworth(data, frequency=fs, cutoff=cutoff)

def moving_mean(data):
    return scared.signal_processing.moving_mean(data, window_size=window_mean)

In [ ]:
prep = [low_pass_filter, moving_mean]

In [ ]:
example = PatternSynchronization(
    ths=ths,
    patterns=[pattern],
    patterns_len=[(0, 6000)],
    score_limit=50,
    preprocesses=prep,
    filtered_output=True,
    frames=[(40000, 55000)]
)

In [ ]:
example.check(5, debug=True)

In [ ]:
example.run("../../Traces/ETS/Telkom_30k_sync1.ets")